# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_surf_lf.ipynb
- template script for analyzing direct arrival waveforms from Sumatra:
- channel LHZ, duration up to 10.5 days

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt 
import numpy as np
import os
import warnings

from obspy.core import read
from pysep import Pysep
from pysep.recsec import plotw_rs
from lib_seis import station_map_and_table
from lib_seis import sumatra_event
from lib_seis import sumatra_waveform_screening
from lib_seis import wf_fft

In [ ]:
warnings.filterwarnings('ignore')                  
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for i, item in enumerate(event):
    print(f'{item} = {event[item]}')
print()

In [ ]:
# download LHZ data, with response removed to your chosen output_unit
# choose your own inputs below

stations   = dict( networks                     = 'G,II,IU',
                   stations                     = '*',
                   locations                    = '*',
                   channels                     = 'LHZ' )

screening  = dict( mindistance_km               = 0,
                   maxdistance_km               = 3000,
                   seconds_before_ref           = 1 * 60 * 60,
                   seconds_after_ref            = 4 * 60 * 60,
                   remove_clipped               = True,
                   remove_insufficient_length   = True,
                   fill_data_gaps               = 0,
                   gap_fraction                 = 0.03,
                   resample_freq                = 1     )

processing = dict( remove_response              = True,
                   output_unit                  = 'VEL',
                   water_level                  = None,
                   pre_filt                     = 'default',
                   detrend                      = True,
                   demean                       = True,
                   taper_percentage             = 0.2   )

output     = dict( log_level                    = 'INFO',
                   plot_files                   = 'map',
                   output_dir                   = 'datawf',
                   sac_subdir                   = '',
                   overwrite_event_tag          = 'sumatra_LHZ_vel',
                   overwrite                    = False )

In [ ]:
# download data

data_dir  = f'{output["output_dir"]}/{output["overwrite_event_tag"]}'
overwrite = f'{output["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**event,**stations,**screening,**processing,**output)
    ps.run()
else:
    print('\nData directory already exists, no data will be downloaded\n')

In [ ]:
# read downloaded LHZ data

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# station map and distances table for the selected station locations
event_path = f'{data_dir}/event.xml'
inv_path = f'{data_dir}/inv.xml'
station_map_and_table(event_path, inv_path)

In [ ]:
# plot the record section using Pyseps's record section plotting tool

plot_info = dict( pysep_path                   = data_dir,
                  sort_by                      = 'distance',
                  scale_by                     = 'normalize',
                  min_period_s                 = 100,
                  max_period_s                 = 1000,
                  preprocess                   = 'st',
                  max_traces_per_rs            = None,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'DEBUG')

plotw_rs(**plot_info)